# Database

In [ ]:
//| export

import { Database } from "@db/sqlite";

export type { Database } from "@db/sqlite";

[Module: null prototype] {
  Database: [class Database],
  SQLBlob: [class SQLBlob],
  SQLITE_SOURCEID: "2024-05-23 13:25:27 96c92aba00c8375bc32fafcdf12429c58bd8aabfcadab6683e35bbb9cdebf19e",
  SQLITE_VERSION: "3.46.0",
  SqliteError: [class SqliteError extends Error],
  Statement: [class Statement],
  isComplete: [Function: isComplete]
}

## Fetch view

In [ ]:
//| export

import { ValidationResult } from "@atproto/lexicon";
import { removeNulls } from "tinychat/utils.ts";

export function fetchView<T>({ db, sql, validate } : { db: Database, sql: string, validate: (v: unknown) => ValidationResult }): T[] {
  console.log("Fetching view using sql:", sql);
  return db.prepare(sql).all().map(r => {
    return Object.keys(r).reduce((acc, k) => {
      if (!k.includes("__")) {
        return Object.assign(acc, { [k]: r[k] });
      }

      const [parent, attr] = k.split("__");

      if (parent === attr) {
        return Object.assign(acc, {
          [parent]: JSON.parse(r[k]),
        });
      }

      return Object.assign(acc, {
        // @ts-ignore yolo
        [parent]: Object.assign(acc[parent] || {}, { [attr]: r[k] }),
      });
    }, {});
  }).map(removeNulls).map((r) => {
    const v = validate(r);
    if (!v.success) {
      console.error("Failed to validate view", v);
    }
    // @ts-ignore yolo
    return v.value;
  }).filter((v) => v);
}

## Bootstrap database

In [ ]:
//| export

import path from "node:path";
import { getProjectRoot } from "tinychat/utils.ts";

const tables: Record<string, string> = {
  users: `
    CREATE TABLE users (
      did TEXT PRIMARY KEY,
      handle TEXT NOT NULL,
      display_name TEXT,
      avatar TEXT,
      description TEXT
    )`,
  servers: `
    CREATE TABLE servers (
      uri TEXT PRIMARY KEY,
      name TEXT NOT NULL,
      creator TEXT NOT NULL,
      created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
      FOREIGN KEY (creator) REFERENCES users(did)
    );`,
  channels: `CREATE TABLE channels (
  id TEXT,
  name TEXT NOT NULL,
  server TEXT NOT NULL,
  latest_message_received_time_us TEXT,
  created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
  PRIMARY KEY (id, server),
  FOREIGN KEY (server) REFERENCES servers(uri)
);`,
  server_memberships: `CREATE TABLE server_memberships (
  user TEXT NOT NULL,
  server TEXT NOT NULL,
  created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
  PRIMARY KEY (user, server),
  FOREIGN KEY (server) REFERENCES servers(uri)
  FOREIGN KEY (user) REFERENCES users(did)
);`,
  messages: `CREATE TABLE messages (
  uri TEXT PRIMARY KEY,
  channel TEXT NOT NULL,
  server TEXT NOT NULL,
  text TEXT NOT NULL,
  sender TEXT NOT NULL,
  created_at DATETIME NOT NULL,
  time_us TEXT NOT NULL,
  FOREIGN KEY (channel, server) REFERENCES channels(id, server),
  FOREIGN KEY (server) REFERENCES servers(uri)
  FOREIGN KEY (sender) REFERENCES users(did)
);
`,
  read_receipts: `CREATE TABLE read_receipts (
  channel TEXT NOT NULL,
  server TEXT NOT NULL,
  user TEXT NOT NULL,
  time_us TEXT NOT NULL,
  PRIMARY KEY (user, channel, server),
  FOREIGN KEY (channel, server) REFERENCES channels(id, server),
  FOREIGN KEY (user) REFERENCES users(did)
);`,
};

let __db: Database | null = null;

export const getDatabase = (
  { reset }: { reset: boolean } = { reset: false },
): Database => {
  if (__db && !reset) {
    return __db;
  }

  const dbPath = Deno.env.get("DB_URL") &&
    path.join(getProjectRoot(), Deno.env.get("DB_URL")!);

  console.log("DB_PATH is", dbPath);

  try {
    __db = new Database(dbPath || ":memory:");
  } catch {
    console.error("Failed to open database at", dbPath);
    console.log("Trying with", Deno.env.get("DB_URL"));
    __db = new Database(Deno.env.get("DB_URL") || ":memory:");
  }

  // WAL please
  __db.exec("pragma journal_mode = WAL");

  const existingTables = __db
    .prepare("SELECT name FROM sqlite_master WHERE type='table'")
    .all<{ name: string }>();

  Object.keys(tables).forEach((table) => {
    if (!existingTables.some((t) => t.name === table)) {
      __db && __db.prepare(tables[table]).run();
    }
  });

  // create triggers and etc
  __db.prepare(`DROP TRIGGER IF EXISTS update_channel_latest_message;`).run();
  __db.prepare(`DROP VIEW IF EXISTS channel_view;`).run();
  __db.prepare(`DROP VIEW IF EXISTS message_view;`).run();
  __db.prepare(`DROP VIEW IF EXISTS server_view;`).run();
  __db.prepare(`DROP VIEW IF EXISTS server_view_with_viewer;`).run();

  __db
    .prepare(
      `CREATE TRIGGER update_channel_latest_message
    AFTER INSERT ON messages
    BEGIN
      UPDATE channels
      SET latest_message_received_time_us = NEW.time_us
      WHERE id = NEW.channel AND (latest_message_received_time_us IS NULL OR NEW.time_us > latest_message_received_time_us);
    END;`,
    )
    .run();

  // simple server view when viewer is not present
  __db
    .prepare(
      `CREATE VIEW server_view AS
       SELECT s.uri, s.name,
        u.did as creator__did,
        u.handle as creator__handle,
        u.display_name as creator__display_name,
        u.avatar as creator__avatar,
        u.description as creator__description,
        json_group_array(
          json_object(
            'id', c.id,
            'name', c.name,
            'server', c.server,
            'latestMessageReceivedTime', c.latest_message_received_time_us
          )
        ) as channels__channels 
      FROM servers s
      INNER JOIN channels c ON c.server = s.uri
      INNER JOIN users u ON u.did = s.creator
      GROUP BY s.uri;
    `,
    )
    .run();

  // server view with viewer
  __db
    .prepare(
      `CREATE VIEW server_view_with_viewer AS
        SELECT 
          s.uri,
          s.name,
          u.did as creator__did,
          u.handle as creator__handle,
          u.display_name as creator__display_name,
          u.avatar as creator__avatar,
          u.description as creator__description,
          c.user as viewer,
          json_group_array(
            json_object(
              'id', c.id,
              'name', c.name,
              'server', c.server,
              'lastMessageReadTime', c.last_message_read_time_us,
              'latestMessageReceivedTime', c.latest_message_received_time_us
            )
          ) as channels__channels
          FROM servers s
          LEFT OUTER JOIN channel_view c ON c.server = s.uri
          INNER JOIN users u ON u.did = s.creator
          GROUP BY s.uri;
      `,
    )
    .run();

  __db
    .prepare(
      `CREATE VIEW channel_view AS
       SELECT c.*, sm.user, COALESCE(rr.time_us, NULL) as last_message_read_time_us
       FROM channels c
       JOIN server_memberships sm ON sm.server = c.server
       LEFT JOIN read_receipts rr ON rr.channel = c.id AND rr.user = sm.user AND rr.server = sm.server;`,
    )
    .run();

  __db
    .prepare(
      `CREATE VIEW message_view AS
       SELECT uri, channel, server, text, sender, created_at as createdAt, time_us, users.did, users.handle, users.display_name as displayName,
              users.avatar, users.description
       FROM messages
       INNER JOIN users ON messages.sender = users.did`,
    )
    .run();

  return __db;
};

In [ ]:
import { assert, assertEquals, assertThrows } from "asserts";

Deno.test("getDatabase", () => {
  const db = getDatabase();
  const ts = db.prepare("SELECT name FROM sqlite_master WHERE type='table'")
    .all<{
      name: string;
    }>();
  assertEquals(ts.length, 6, "got 6 tables");
  assert(ts.some((t) => t.name === "users"), "got users table");
  assert(ts.some((t) => t.name === "servers"), "got servers table");
  assert(ts.some((t) => t.name === "channels"), "got channels table");
  assert(
    ts.some((t) => t.name === "server_memberships"),
    "got server_memberships table",
  );
  assert(ts.some((t) => t.name === "messages"), "got messages table");
  assert(ts.some((t) => t.name === "read_receipts"), "got read_receipts table");

  const views = db
    .prepare("SELECT name FROM sqlite_master WHERE type='view'")
    .all<{
      name: string;
    }>();

  assertEquals(views.length, 4, "got 4 views");
  assert(views.some((v) => v.name === "channel_view"), "got channel_view");
  assert(views.some((v) => v.name === "message_view"), "got message_view");
  assert(views.some((v) => v.name === "server_view"), "got server_view");
  assert(
    views.some((v) => v.name === "server_view_with_viewer"),
    "got server_view_with_viewer",
  );
});

Deno.test("test keys for memberships", () => {
  const db = getDatabase();

  assertThrows(() => {
    db.prepare(
      "INSERT INTO server_memberships (server, user) VALUES (:server, :user)",
    ).run({
      server: "1",
      user: "2",
    });
  }, "foreign key constraints");

  db.prepare(
    "INSERT INTO users (did, handle, display_name) VALUES (:did, :handle, :display_name)",
  ).run({
    did: "1",
    handle: "user1",
    display_name: "User 1",
  });

  db.prepare(
    "INSERT INTO servers (uri, name, creator) VALUES (:uri, :name, :creator)",
  ).run({
    uri: "1",
    name: "Server 1",
    creator: "1",
  });

  db.prepare(
    "INSERT INTO server_memberships (server, user) VALUES (:server, :user)",
  ).run({
    server: "1",
    user: "1",
  });

  // try select in

  assert(
    db
      .prepare(
        `SELECT * FROM server_memberships WHERE user IN (${
          ["1"]
            .map((id) => `'${id}'`)
            .join(",")
        })`,
      )
      .all().length === 1,
  );

  // try inserting duplicate membership

  assertThrows(() => {
    db.prepare(
      "INSERT INTO server_memberships (server, user) VALUES (:server, :user)",
    ).run({
      server: "1",
      user: "1",
    });
  }, "duplicate server membership");
});